# 1. 데이터셋 청크 단위로 전처리

In [4]:
!pip install pymupdf pandas tabulate

In [5]:
import fitz
import pandas as pd
import re
import statistics

def pdf_parser(pdf_path):
    doc = fitz.open(pdf_path)
    final_chunks = []
    current_chunk = None
    last_main_header_text = ""

    for page_num, page in enumerate(doc, 1):
        all_elements = []

        # 페이지에서 이미지, 테이블, 텍스트 블록 추출
        image_list = page.get_images(full=True)
        tables = page.find_tables()
        page_blocks = page.get_text("dict", flags=fitz.TEXTFLAGS_SEARCH)["blocks"]

        # 테이블 bbox 저장하여 테이블 내 텍스트를 일반 텍스트로 중복 처리하지 않도록 함
        table_bboxes = [fitz.Rect(t.bbox) for t in tables]

        # 폰트 크기 수집하여 가장 흔한 폰트 크기를 기본 폰트로 두고 제목과 본문 구분
        font_sizes = [span["size"] for block in page_blocks if block["type"] == 0 for line in block["lines"] for span in line["spans"]]
        base_font_size = statistics.mode(font_sizes) if font_sizes else 10.0

        # 텍스트 블록 순회하며 테이블에 속하지 않는 텍스트만 all_elements에 추가
        for block in page_blocks:
            if block['type'] == 0:
                block_bbox = fitz.Rect(block['bbox'])
                if not any(block_bbox.intersects(t_bbox) for t_bbox in table_bboxes):
                    all_elements.append({'type': 'text', 'bbox': block_bbox, 'data': block})

        # 이미지 정보 all_elements에 추가
        for img_info in image_list:
            width, height = img_info[2], img_info[3]
            bbox = page.get_image_bbox(img_info)
            all_elements.append({'type': 'image', 'bbox': bbox, 'data': f"[Image Found on Page {page_num}]"})

        # 테이블 정보 pandas DataFrame -> Markdowm 형식으로 all_elements에 추가
        for i, table in enumerate(tables):
            df = table.to_pandas()
            if not df.empty:
                all_elements.append({'type': 'table', 'bbox': table_bboxes[i], 'data': df.to_markdown(index=False)})

        # 페이지의 모든 요소를 y좌표 기준으로 정렬하여 시각적 순서대로 요소 처리
        all_elements.sort(key=lambda x: x['bbox'].y0)

        # 정렬된 요소들 처리하며 청크로 묶는 과정
        for elem in all_elements:
            elem_type = elem['type']

            # 이미지 or 테이블일 경우 현재 청크에 내용 추가
            if elem_type == 'image' or elem_type == 'table':
                if current_chunk:
                    current_chunk['content'].append(elem['data'])
                    if elem_type == 'image': current_chunk['has_image'] = True
                elif not final_chunks:
                    current_chunk = {"header": "Initial Content", "content": [elem['data']], "start_page": page_num, "type": "section", "has_image": (elem_type == 'image')}
                else:
                    final_chunks[-1]['content'].insert(0, elem['data'])
                    if elem_type == 'image': final_chunks[-1]['has_image'] = True
                continue

            # 요소가 텍스트인 경우 제목인지 본문인지 판단
            block = elem['data']
            raw_text = " ".join([span["text"] for line in block["lines"] for span in line["spans"]]).strip()
            block_text = raw_text.replace("\n", " ")
            if not block_text:
                continue

            # 폰트 크기와 텍스트 패턴을 기존으로 내용 유형 결정
            content_type = "paragraph"  # 본문
            first_span = block["lines"][0]["spans"][0]
            if first_span["size"] > base_font_size * 1.5: content_type = "header_level_1"  # 대제목
            elif first_span["size"] > base_font_size * 1.2: content_type = "header_level_2"  # 소제목
            if re.match(r"^\d+\.", block_text): content_type = "header_level_1"
            elif re.match(r"^\d+-\d+\.", block_text): content_type = "header_level_2"

            # 내용 유형이 제목일 경우 이전에 작업하던 청크를 마무리 지음
            if content_type.startswith("header"):
                if current_chunk:
                    current_chunk["content"] = "".join(current_chunk["content"]).strip()
                    if current_chunk["content"]:
                        final_chunks.append(current_chunk)

                # 새로운 청크 시작
                header_text = block_text
                if content_type == "header_level_1":
                    last_main_header_text = block_text

                # 소제목의 경우 대제목 - 소제목 형태로 제목 구성
                elif last_main_header_text:
                    header_text = f"{last_main_header_text} - {block_text}"

                current_chunk = {"header": header_text, "content": [], "start_page": page_num, "type": "section", "has_image": False}

            # 내용 유형이 본문인 경우, 현재 청크의 내용에 추가
            elif current_chunk:
                current_chunk['content'].append(block_text)

    # 모든 페이지 처리가 끝난 후 마지막 작업 중이던 청크 마무리 지음
    if current_chunk:
        current_chunk["content"] = "".join(current_chunk["content"]).strip()
        if current_chunk["content"]:
            final_chunks.append(current_chunk)

    doc.close()
    return final_chunks

# 코드 실행
if __name__ == "__main__":
    pdf_file = "/content/조수기 데모용 미니 데이터셋-shell.pdf"
    final_chunks = pdf_parser(pdf_file)

    print(f"총 {len(final_chunks)}개의 유효한 섹션으로 병합되었습니다.\n")
    for i, chunk in enumerate(final_chunks):
        print(f"--- 섹션 {i+1} ---")
        if chunk.get("has_image", False):
            print("  📸 이미지가 포함된 섹션입니다.")
        print(f"  📖 대제목: {chunk.get('header', 'N/A')}")
        print(f"  📄 시작 페이지: {chunk['start_page']}")
        print(f"  📝 내용 미리보기: {chunk['content'][:200].strip()}...")
        print("-" * 25)

총 27개의 유효한 섹션으로 병합되었습니다.

--- 섹션 1 ---
  📸 이미지가 포함된 섹션입니다.
  📖 대제목: 0.   조수기 작동원리
  📄 시작 페이지: 1
  📝 내용 미리보기: 조수기는 기본적으로 해수를 끓여 이 수증기를 냉각시켜 물을 만드는 기기이다. 그런데 이 물을 어떻게 끓여 수증기를 만들고 냉각시켜 물을 만드느냐가 조수기의 성능을 좌우하게 된다. 우리들이 일상적으로 생활하는 공간의 압력을 대기압이라고 하는데, 이 대기압에서 물이 끓는 온도는 100℃이다. 이와 같이 100℃에서 물을 끓이는데 많은 열이 필요하나 선박에서는...
-------------------------
--- 섹션 2 ---
  📖 대제목: 3.   조수기 운전 및 정지 - 3-1.  Starting up (KE Type)
  📄 시작 페이지: 3
  📝 내용 미리보기: 1)   조수기를 운전하기 전에 다음 밸브는 Close되어 있어야 한다.◆  Heat Exchanger의 Jacket Cool. Water용 입.출구 밸브◆  Vacuum Breaker Valve◆  Distillate Pump의 출구변◆  Heat Exchanger의 Feed Water 입구변◆  Bottom Blow Valve2)   Ejector Pu...
-------------------------
--- 섹션 3 ---
  📖 대제목: 3.   조수기 운전 및 정지 - 3-2.  Stopping
  📄 시작 페이지: 4
  📝 내용 미리보기: 1) Salinity Indicator의 전원을 OFF한다.2) Heat exchanger 가열용 Jacket Cool. Water  By-pass V/V를 Open하고 입출구변을 Close한다.3) Distillate Pump를 정지하고 출구변을 Close한다.4) Heat Exchanger의 Feed Water V/V를 Close – Heat Excha...
-------------------------
--- 섹션 4 ---
  📖 대제목: 3

# 2. FAISS를 이용해 벡터 인덱스 생성

In [1]:
!pip install sentence-transformers faiss-gpu-cu12

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 임베딩 모델 로드
embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B") # snunlp/KR-SBERT-V40K-klueNLI-augSTS
embedding_model.to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 32768, 'do_lower_case': False, 'architecture': 'Qwen3Model'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': True, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
# 1. 문서 파싱
pdf_file = "/content/조수기 데모용 미니 데이터셋-shell.pdf"
chunks = pdf_parser(pdf_file)
documents = [
    f"제목: {chunk['header']}, 내용: {chunk['content']}" for chunk in chunks
]

# 2. 문서 임베딩
print("문서 임베딩을 시작합니다...")
embeddings = embedding_model.encode(documents, convert_to_tensor=True, device=device)
print("임베딩 완료!")

# 3. FAISS로 벡터 검색 엔진(인덱스) 구축
d = embeddings.shape[1]
res = faiss.StandardGpuResources()
index_flat = faiss.IndexFlatL2(d)
gpu_index = faiss.index_cpu_to_gpu(res, 0, index_flat)
gpu_index.add(embeddings.cpu().numpy())

문서 임베딩을 시작합니다...
임베딩 완료!


#3. LLM 프롬프트 작성 및 입력

In [7]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai
import faiss.contrib.torch_utils


def search_and_answer(query, embedding_model, top_k=3):

    # 1. 사용자 질문 임베딩
    query_vector = embedding_model.encode([query], prompt_name='query', convert_to_tensor=True, device=device)

    # 2. FAISS 인덱스에서 가장 관련 있는 섹션 K개 검색
    D, I = gpu_index.search(query_vector, top_k)

    # 3. 검색된 관련 문서 내용 취합
    retrieved_docs = [chunks[i] for i in I[0]]

    context = ""
    for i, doc in enumerate(retrieved_docs):
        context += f"--- 관련 문서 #{i+1} (제목: {doc['header']}) ---\n"
        context += doc['content']
        context += "\n\n"

    # 4. 프롬프트 생성
    prompt = f"""
당신은 '조수기 기술 매뉴얼' 전문가입니다.
주어진 관련 문서를 바탕으로 사용자의 질문에 대해 한국어로 답변해주세요.

--- 관련 문서 ---
{context}
--- 질문 ---
{query}

답변:
"""

    print("--- 최종 프롬프트 ---")
    print(prompt)

    GOOGLE_API_KEY=""

    genai.configure(api_key=GOOGLE_API_KEY)

    LLM = genai.GenerativeModel('gemini-flash-latest')

    print("---모델 답변---")

    response = LLM.generate_content(prompt)

    print(response.text)


# 5. LLM 실행
user_question = "조수량이 너무 적게 나옵니다. 어떤 순서로 확인해봐야 할까요?"
search_and_answer(user_question, embedding_model)

--- 최종 프롬프트 ---

당신은 '조수기 기술 매뉴얼' 전문가입니다.
주어진 관련 문서를 바탕으로 사용자의 질문에 대해 한국어로 답변해주세요.

--- 관련 문서 ---
--- 관련 문서 #1 (제목: 4.   운전 (DX-α 시리즈, 재킷 냉각수 이용) - 4-5. 운전 중 주의사항 (Attention during operation)) ---
1)생산량 조절•   담수 생산량은 재킷 냉각수 유량 조절에 따라 결정•   청결한 상태에서는 정격 이상 생산도 가능하나, 과잉 생산 시 스케일 형성 위험→  정격 이하 운전 권장2)콘덴서•   증기 전량 응축 위해서는 냉각해수는  가능한 차갑게  유지•   냉각해수 입출구 온도차 반드시 기록 → 냉각수량 조절 근거로 사용3) 냉각해수 유량 문제:•   과잉 공급  → 고속 유동으로  부식(침식)  위험 ↑•   부족 공급  → 냉각 불충분 → 담수 생산량 감소4)냉각수량 계산식:Q=D×24.5t1 − t2Q = \frac{D \times 24.5}{t1 - t2}Q=t1 − t2D×24.5•   D = 담수 생산량 (톤/일)•   t1 = 콘덴서 입구 온도•   t2 = 콘덴서 출구 온도•   Q = 콘덴서 냉각수 유량정기적으로 운전 데이터를 기록하면, FWG의  이상 징후 조기 발견  가능하다.5)점검(Inspection)•   주기 : 예를 들어  연 2회  정기 점검.•   내용 :o   분리기 상부 커버와 열교환기 하부 커버를 열어  튜브 내부 스케일  및  내부 코팅 박리(exfoliation)  여부 점검.o   콘덴서 커버를 열어  냉각 튜브 슬러지/이물 부착  여부 점검.o   펌프 는 정기적으로 점검·세척하고,  부식 부품 은 예비품으로 교체.

--- 관련 문서 #2 (제목: 3.   조수기 운전 및 정지 - 3-4.  조수량의 조절) ---
1)   조수량의 조절은 Heat Exchanger로 가는 Jacket Cool. F.W의 양을 가감함으로써가능하다. Jacket Cool. F.W 온도가